In [1]:
import numpy as np # linear algebra
import numpy.linalg as ln
import pandas as pd # data processing
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import random
import statistics

# Importing the Datasets CSV

dataset_customers = "C:/Users/Alessia/Documents/GitHub/H-M-Customer-Segmentation-CRISP-DM/Dataset/customers.csv"
dataset_articles = "C:/Users/Alessia/Documents/GitHub/H-M-Customer-Segmentation-CRISP-DM/Dataset/articles.csv"
dataset_transactions = "C:/Users/Alessia/Documents/GitHub/H-M-Customer-Segmentation-CRISP-DM/Dataset/transactions_train.csv"

# Read Datasets
customers = pd.read_csv(dataset_customers)

articles = pd.read_csv(dataset_articles)

transactions = pd.read_csv(dataset_transactions)

print("loading finished")

loading finished


In [2]:
# Import CPT
import os
import sys
sys.path.insert(0, os.path.abspath('C:/Users/Alessia/Documents/CPT-master/CPT-master/modules'))
import CPT
import PredictionTree
from CPT import *

In [3]:
# Instantiate CPT model
# model = CPT()
# data,target = model.load_files("C:/Users/Alessia/Documents/GitHub/H-M-Sales-CRISP-DM-Analysis/trainprova.csv","C:/Users/Alessia/Documents/GitHub/H-M-Sales-CRISP-DM-Analysis/testprova.csv")
# print(data)
# model.train(data)

In [4]:
# Predictions using CPT, using the last 5 data in each sequence, and I should predict the next 3 items

# predictions = model.predict(data,target,5,3)
# print(predictions)

In [5]:
#### EXPLORATORY ANALYSIS

# Number of features and rows in Customers 
c_rows = customers.shape[0]
c_features = customers.shape[1]
print (c_rows, c_features)

# Number of features and rows in Articles
a_rows = articles.shape[0]
a_features = articles.shape[1]
print (a_rows, a_features)

# Number of features and rows in Transactions
t_rows = transactions.shape[0]
t_features = transactions.shape[1] 
print(t_rows, t_features)


1371980 7
105542 25
31788324 5


In [6]:
# Getting knowledge about updatedness of data
print(transactions.iloc[1]['t_dat'], transactions.iloc[-1]['t_dat'])

2018-09-20 2020-09-22


In [7]:
# Exploring the features of each dataset
print("Customers features: " + str(c_features))
print(customers.dtypes)
print("###########################")
print("Articles features: " + str(a_features))
print(articles.dtypes)
print("###########################")
print("Transactions features: " + str(t_features))
print(transactions.dtypes)


Customers features: 7
customer_id                object
FN                        float64
Active                    float64
club_member_status         object
fashion_news_frequency     object
age                       float64
postal_code                object
dtype: object
###########################
Articles features: 25
article_id                       int64
product_code                     int64
prod_name                       object
product_type_no                  int64
product_type_name               object
product_group_name              object
graphical_appearance_no          int64
graphical_appearance_name       object
colour_group_code                int64
colour_group_name               object
perceived_colour_value_id        int64
perceived_colour_value_name     object
perceived_colour_master_id       int64
perceived_colour_master_name    object
department_no                    int64
department_name                 object
index_code                      object
index_name   

In [8]:
# How many different customers postal codes do we have?
print(len(customers['postal_code'].unique()))

352899


In [9]:
# Check if there are customer rows where some features are null
print(customers.isna().sum())

# Drop all columns where age is NaN
customers = customers[customers["age"].notna()]

# Fill the NaN values in Active
customers['Active'] = customers['Active'].fillna(0)

customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
postal_code                    0
dtype: int64


In [10]:
# Replace all the occurrences of NONE with None

customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace(['NONE'],'None')

In [11]:
# Drop all columns where "club_member_status" and "fashion_news_frequency" are NaN
customers = customers[customers["club_member_status"].notna()]
customers = customers[customers["fashion_news_frequency"].notna()]

In [12]:
# Extract non null values from "club_member_status" column and use them to fill the NaN values of the same column

# values = customers['club_member_status'].value_counts(dropna=True)

# a_v = customers[customers['club_member_status'].notna()]
# active_values=a_v['club_member_status'].unique()
# customers['club_member_status'] = customers['club_member_status'].fillna(np.random.choice(active_values.tolist()))
# print(customers['club_member_status'].unique())

Che cosa sto facendo sostanzialmente?
Ho visto che per gli attributi "fashion_news_frequency" e "club_member_status" i valori nulli erano pochi rispetto alle totali istanze del dataset, per questo decido di dropparle dal dataset. Per quanto riguarda invece l'attributo FN, le istanze con i valori nulli sono molte rispetto al totale, quindi non posso rimuovere tutte quelle che non possiedono un valore per FN. Per questo motivo decido di non prendere in considerazione questa feature per la mia analisi.

In [13]:
# Replace NaN with 0 
customers['FN'] = customers['FN'].fillna(0)

In [14]:
# Check the possible values for each of the following attributes
print(customers['Active'].unique(), customers['club_member_status'].unique(),customers['fashion_news_frequency'].unique(),customers['FN'].unique())

[0. 1.] ['ACTIVE' 'PRE-CREATE' 'LEFT CLUB'] ['None' 'Regularly' 'Monthly'] [0. 1.]


In [15]:
# At this point i have 1338570 homogeneous records for Customers
print(customers.notna().sum())

customer_id               1338570
FN                        1338570
Active                    1338570
club_member_status        1338570
fashion_news_frequency    1338570
age                       1338570
postal_code               1338570
dtype: int64


In [16]:
# Check the transactions
print(transactions.isna().sum())

# The records for the transactions are all consistent

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64


In [17]:
# I split the customers
customers_1 = customers[customers['customer_id'] < '669286']
customers_2 = customers[customers['customer_id'] >= '669286']

In [18]:
# Merging customers and transactions
customers_transactions_1 = pd.merge(customers_1, transactions, on='customer_id')
customers_transactions_2 = pd.merge(customers_2, transactions, on='customer_id')

In [19]:
customers_transactions = customers_transactions_1.append(customers_transactions_2)

In [20]:
# Print unique values of customer_id in the whole dataframe customers_transactions
print(len(customers_transactions["customer_id"].unique()))

# Double check of the number of customers without any transaction. There are 9583 customers without any transaction
special_customers = customers[~customers["customer_id"].astype(str).isin(customers_transactions["customer_id"].astype(str).unique())]
print(len(special_customers))

1328987


KeyboardInterrupt: 

In [ ]:
# Split transactions dataframe because it's too large
#trans_2018 = transactions[transactions['t_dat'] < "2019-01-01"]
#trans_2019 = transactions[(transactions['t_dat'] < "2020-01-01") & (transactions['t_dat'] >= "2019-01-01")]
#trans_2020 = transactions[transactions['t_dat'] >= "2020-01-01"]

#print(len(trans_2018),len(trans_2019),len(trans_2020))

# Check if the split was successful
#print(trans_2018.iloc[1]['t_dat'], trans_2018.iloc[-1]['t_dat'])
#print(trans_2019.iloc[1]['t_dat'], trans_2019.iloc[-1]['t_dat'])
#print(trans_2020.iloc[1]['t_dat'], trans_2020.iloc[-1]['t_dat'])

# Merge dataframes Customers and Transactions
# customers_transactions_2018 = pd.merge(customers, trans_2018, on='customer_id')

# customers_transactions_2019 = pd.merge(customers_transactions_2018, trans_2019, on='customer_id')
# print(customers_transactions_2019.head())

In [ ]:
print(customers_transactions["sales_channel_id"].unique())

In [ ]:
# Finding number of articles purchased by each Customer in 2018
df_2018 = customers_transactions.query('t_dat < "2019-01-01"')

# Filter out labels of interest
df_2018 = df_2018.filter(['customer_id', 'article_id'])
df_2018.rename(columns = {'article_id':'tot_articles_2018'}, inplace = True)

In [ ]:
# Finding number of articles purchased by each Customer in 2019
df_2019 = customers_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
 
# Filter out labels of interest
df_2019 = df_2019.filter(['customer_id', 'article_id'])
df_2019.rename(columns = {'article_id':'tot_articles_2019'}, inplace = True)

In [ ]:
# Finding number of articles purchased by each Customer in 2020
df_2020 = customers_transactions.query('t_dat >= "2020-01-01"')
 
# Filter out labels of interest
df_2020 = df_2020.filter(['customer_id', 'article_id'])
df_2020.rename(columns = {'article_id':'tot_articles_2020'}, inplace = True)

In [ ]:
# Merge these new features into Customers dataset
customers = pd.merge(customers,df_2018.groupby('customer_id').count(), on='customer_id')
customers = pd.merge(customers,df_2019.groupby('customer_id').count(), on='customer_id')
customers = pd.merge(customers,df_2020.groupby('customer_id').count(), on='customer_id')

In [ ]:
# NEW ######
# Merge with unique articles

"""
print(customers.head())

df_2018_unique = df_2018.filter(['customer_id', 'tot_articles_2018'])
df_2018_unique = df_2018_unique.groupby('customer_id').apply(lambda x: x.tot_articles_2018.nunique())

df_2019_unique = df_2019.filter(['customer_id', 'tot_articles_2019'])
df_2019_unique = df_2019_unique.groupby('customer_id').apply(lambda x: x.tot_articles_2019.nunique())

df_2020_unique = df_2020.filter(['customer_id', 'tot_articles_2020'])
df_2020_unique = df_2020_unique.groupby('customer_id').apply(lambda x: x.tot_articles_2020.nunique())    


customers = pd.merge(customers,df_2018_unique, on='customer_id', how='left')
customers = pd.merge(customers,df_2019_unique, on='customer_id', how='left')
customers = pd.merge(customers,df_2020_unique, on='customer_id',how='left') 


#customers.rename(columns = {'tot_articles_2018_x':'unique_articles_2018'}, inplace = True)
#customers.rename(columns = {'tot_articles_2019_x':'unique_articles_2019'}, inplace = True)
#customers.rename(columns = {'tot_articles_2020_x':'unique_articles_2020'}, inplace = True)"""

In [ ]:
# Merge average articles 2018,2019,2020
customers = pd.merge(customers,df_2018.groupby('customer_id').count()/3, on='customer_id')
customers = pd.merge(customers,df_2019.groupby('customer_id').count()/12, on='customer_id')
customers = pd.merge(customers,df_2020.groupby('customer_id').count()/9, on='customer_id')

# rename

# Rename
customers.rename(columns = {'tot_articles_2018_x':'tot_articles_2018'}, inplace = True)
customers.rename(columns = {'tot_articles_2019_x':'tot_articles_2019'}, inplace = True)
customers.rename(columns = {'tot_articles_2020_x':'tot_articles_2020'}, inplace = True)
customers.rename(columns = {'tot_articles_2018_y':'avg_articles_2018'}, inplace = True)
customers.rename(columns = {'tot_articles_2019_y':'avg_articles_2019'}, inplace = True)
customers.rename(columns = {'tot_articles_2020_y':'avg_articles_2020'}, inplace = True)


In [ ]:
print(customers.head())
print(customers.shape)

In [ ]:
# Find the number of transactions of type 1 for each customer
trans_type1 = customers_transactions.query('sales_channel_id == 1')
trans_type1 = trans_type1.filter(['customer_id', 'sales_channel_id'])
trans_type1.rename(columns = {'sales_channel_id':'sales_channel_1'}, inplace = True)
print(trans_type1.groupby('customer_id').count())

In [ ]:
# Find the number of transactions of type 2 for each customer
trans_type2 = customers_transactions.query('sales_channel_id == 2')
trans_type2 = trans_type2.filter(['customer_id', 'sales_channel_id'])
trans_type2.rename(columns = {'sales_channel_id':'sales_channel_2'}, inplace = True)
print(trans_type2.groupby('customer_id').count())

In [ ]:
# Merge these new features into customers table

customers = pd.merge(customers, trans_type1.groupby('customer_id').count(), on='customer_id')
print(customers.shape)
customers = pd.merge(customers,trans_type2.groupby('customer_id').count(), on='customer_id')
print(customers.shape)

In [ ]:
print(customers.head())

In [ ]:
# Merging Customers with Articles 
# customers_articles = pd.merge(pd.read_csv(dataset_customers), pd.read_csv(dataset_articles), on ='customer_id')
print(customers_transactions.head())

In [ ]:
# Need to check some variables about the price column

# Mean of the data

# mean = statistics.fmean(transactions["price"])
# print(mean)


In [ ]:
# Square deviations

# stdev = statistics.stdev(transactions["price"])
# print(stdev)

In [ ]:
# Variance
# variance = statistics.variance(transactions["price"])
# print(variance)

In [ ]:
# Norm
# norm = ln.norm(transactions["price"])
#print(norm)

# Mean 0.027829273856993762
# Std-deviation 0.019181128054793647
# Variance 0.00036791567345439193
# Norm 190.56357821702613

In [ ]:
# I have checked that my values are standardized, but the values are not satisfactory.
# So I decide to scale my values multiplying them for 10^3 in order to make them prices that are more realistic.
# I have that my lowest value is 0.01 and my biggest is 591.52
#transactions["price"] = transactions["price"] * (pow(10,3)) 


# Print the new min and max values, and the rows where they appear.
print(transactions["price"].max(), transactions["price"].idxmax(),  transactions["price"].min(),transactions["price"].idxmin())

In [ ]:
##### Pick the important columns from transactions and then merge them with customers table
am_2018 = transactions.query('t_dat < "2019-01-01"')

#Filter out labels of interest
am_2018 = am_2018.filter(['customer_id', 'price'])
am_2018.rename(columns = {'price':'tot_amount_2018'}, inplace = True)

In [ ]:
##### Pick the important columns from transactions and then merge them with customers table
am_2019 = transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')

#Filter out labels of interest
am_2019 = am_2019.filter(['customer_id', 'price'])
am_2019.rename(columns = {'price':'tot_amount_2019'}, inplace = True)

In [ ]:
##### Pick the important columns from transactions and then merge them with customers table
am_2020 = transactions.query('t_dat >= "2020-01-01"')

#Filter out labels of interest
am_2020 = am_2020.filter(['customer_id', 'price'])
am_2020.rename(columns = {'price':'tot_amount_2020'}, inplace = True)

In [ ]:
# Merge these new tot_amount features into customers table
customers = pd.merge(customers, am_2018.groupby('customer_id').sum(), on='customer_id')
customers = pd.merge(customers, am_2019.groupby('customer_id').sum(), on='customer_id')
customers = pd.merge(customers, am_2020.groupby('customer_id').sum(), on='customer_id')

In [ ]:
# Merge these new avg amounts features into customers table
customers = pd.merge(customers, am_2018.groupby('customer_id').sum()/3, on='customer_id')
customers = pd.merge(customers, am_2019.groupby('customer_id').sum()/12, on='customer_id')
customers = pd.merge(customers, am_2020.groupby('customer_id').sum()/9, on='customer_id')

In [ ]:
# Rename columns in customers
customers.rename(columns = {'tot_amount_2018_y':'avg_amount_2018'}, inplace = True)
customers.rename(columns = {'tot_amount_2019_y':'avg_amount_2019'}, inplace = True)
customers.rename(columns = {'tot_amount_2020_y':'avg_amount_2020'}, inplace = True)
customers.rename(columns = {'tot_amount_2018_x':'tot_amount_2018'}, inplace = True)
customers.rename(columns = {'tot_amount_2019_x':'tot_amount_2019'}, inplace = True)
customers.rename(columns = {'tot_amount_2020_x':'tot_amount_2020'}, inplace = True)
print(customers.head())

In [ ]:
# Infer sex of the customers thanks to the purchased articles

# Analyze possible categories
print(articles["index_group_name"].unique())
print(articles["index_group_no"].unique())

In [ ]:
# We find how many articles we have divided for categories
sex_category = articles[["index_group_no", "index_group_name"]].reset_index()
display(sex_category["index_group_name"].value_counts())

In [ ]:
# We find the list of categories of articles and we plot the percentages of articles categories
sex_category_list = sex_category["index_group_name"].value_counts().index.to_list()
plt.figure(figsize=(5, 5))
plt.rcParams["font.size"] = 12
plt.pie(sex_category["index_group_name"].value_counts().sort_values(ascending=False), 
        labels = sex_category_list, startangle = 90, counterclock=False, autopct="%1.1f%%")
plt.show

In [ ]:
# I rename the index group as sex attribute
articles_category_df = pd.DataFrame(articles[["article_id", "index_group_no"]])
articles_category_df.columns = ["article_id", "sex_attribute"]
articles_category_df

In [ ]:
# I merge these sex attributes with the dataframe customers_transactions
customers_transactions = pd.merge(customers_transactions, articles_category_df, on = "article_id", how = "left")

In [ ]:
cust_sex = customers_transactions[["customer_id", "sex_attribute", "article_id"]].groupby(["customer_id","sex_attribute"]).count().unstack()
cust_sex.columns = ["Woman", "Young", "Man", "Have-kids", "Sports-person"]

In [ ]:
# Fill NaN values
cust_sex = cust_sex.fillna(0)

In [ ]:
# Merge these attributes into the customers dataset 
customers = pd.merge(customers, cust_sex, on ='customer_id')
customers.head()

In [ ]:
print(customers.shape)

In [ ]:
# Analysis on the age ranges
print(customers["age"].min(),customers["age"].max())

In [ ]:
# NON FUNZIONA

#customers.loc[customers['age'] >= 16 & customers['age'] < 36,customers['age']] = "young adult"
#customers['age'] = customers['age'].astype(str)
#print(customers.dtypes)
#customers['age'] = customers['age'].mask(customers['age'] >= 66, "senior", inplace=True)
#customers['age'] = customers['age'].mask((customers['age'] >= 16) & (customers['age'] < 36), "young adult", inplace=True)
#customers['age'] = customers['age'].mask((customers['age'] >= 36) & (customers['age'] < 51), "adult", inplace=True)
#customers['age'] = customers['age'].mask((customers['age'] >= 51) & (customers['age'] < 66), "mid age adult", inplace=True)
#print(customers.head())

In [ ]:
# Replace continuous ages with discrete ages groups
#ages_group = customers.query('age >= 16 & age < 36')
#ages_group = ages_group.filter(['customer_id', 'age'])
#ages_group.loc[(ages_group['age']>= 16) & (ages_group['age'] < 36) , 'age'] = 'young adult'

#ages_group = customers.query('age >= 36 & age < 51')
#ages_group = ages_group.filter(['customer_id', 'age'])
#ages_group.loc[(ages_group['age']>= 36) & (ages_group['age'] < 51) , 'age'] = 'adult'

#ages_group = customers.query('age >= 51 & age < 66')
#ages_group = ages_group.filter(['customer_id', 'age'])
#ages_group.loc[(ages_group['age']>= 51) & (ages_group['age'] < 66) , 'age'] = 'mid aged adult'

#ages_group = customers.query('age >= 65')
#ages_group = ages_group.filter(['customer_id', 'age'])
#ages_group.loc[ages_group['age']>= 65 , 'age'] = 'senior'

In [ ]:
# Check ages before the transformation
customers.head()

In [ ]:
# I assign a numeric category for each age range

#customers.loc[(customers['age']>= 16) & (customers['age']< 36) , 'age'] = 1
#customers.loc[(customers['age'] >= 36) & (customers['age'] < 51), 'age'] = 2
#customers.loc[(customers['age'] >= 51) & (customers['age'] < 66) , 'age'] = 3
#customers.loc[customers['age']>= 66, 'age'] = 4 

In [ ]:
# Check numeric categories
customers.head()

In [ ]:
# I replace numeric categories with categorical attributes
#customers['age'].replace({'young adult':1 },inplace=True)
#customers['age'].replace({'adult': 2},inplace=True)
#customers['age'].replace({'mid aged adult': 3},inplace=True)
#customers['age'].replace({'senior':4 },inplace=True)

In [ ]:
# Drop postal code 

# I drop postal code because it doesn't tell us anything about the behaviour of the customer in the field of shopping. It's just a string
customers= customers.drop('postal_code',1)

In [ ]:
# Need to find the garment and the color more purchased for each year
transac_drop_df = pd.DataFrame(transactions[['customer_id', 'article_id', 't_dat']]).reset_index()
articles_drop_df = pd.DataFrame(articles[['article_id', 'colour_group_code', 'garment_group_no',"perceived_colour_master_id", 'section_no']])

In [ ]:
articles_transactions =  pd.merge(transac_drop_df, articles_drop_df, on='article_id', how="left")
print(articles_transactions.head())

In [ ]:
# Get number of articles for each customer that are purchased more than one time
"""customers['count_purchased_more_than_one']=''
#df=customers_transactions.filter(['customer_id', 'article_id'])
df = customers_transactions[customers_transactions.groupby(['customer_id','article_id'])['article_id'].transform('size').lt(2)]

# unique articles 23546342
# 7925230

## Articles per customer bought only one time 
df['new_col']=''
df['new_col'] = df.groupby(["customer_id",'article_id']).count()
print(df)"""

In [ ]:
# Need to find the counts for each colour
## POSSIBLE COLOURS ARE 50, POSSIBLE GARMENTS ARE 21
customer_articles_2018 = articles_transactions.query('t_dat < "2019-01-01"')
customer_articles_2018 = customer_articles_2018.filter(['customer_id', 'colour_group_code'])
customers = pd.merge(customers,customer_articles_2018.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'colour_group_code':'count_colours_2018'}, inplace = True)
print(customers.head())

In [ ]:
customer_articles_2019 = articles_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
customer_articles_2019 = customer_articles_2019.filter(['customer_id', 'colour_group_code'])
customers = pd.merge(customers,customer_articles_2019.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'colour_group_code':'count_colours_2019'}, inplace = True)
print(customers.head())

In [ ]:
customer_articles_2020 = articles_transactions.query('t_dat >= "2020-01-01"')
customer_articles_2020 = customer_articles_2020.filter(['customer_id', 'colour_group_code'])
customers = pd.merge(customers,customer_articles_2020.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'colour_group_code':'count_colours_2020'}, inplace = True)
print(customers.head())

In [ ]:
customer_garments_2018 = articles_transactions.query('t_dat < "2019-01-01"')
customer_garments_2018 = customer_garments_2018.filter(['customer_id', 'garment_group_no'])
customers = pd.merge(customers,customer_garments_2018.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'garment_group_no':'count_garments_2018'}, inplace = True)

In [ ]:
customer_garments_2019 = articles_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
customer_garments_2019 = customer_garments_2019.filter(['customer_id', 'garment_group_no'])
customers = pd.merge(customers,customer_garments_2019.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'garment_group_no':'count_garments_2019'}, inplace = True)

In [ ]:
customer_garments_2020 = articles_transactions.query('t_dat >= "2020-01-01"')
customer_garments_2020 = customer_garments_2020.filter(['customer_id', 'garment_group_no'])
customers = pd.merge(customers,customer_garments_2020.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'garment_group_no':'count_garments_2020'}, inplace = True)

In [ ]:
#customers_articles_2018 = customers_articles_2018.groupby(['customer_id'])['colour_group_code'].agg(pd.Series.mode).to_frame()

# Find the mode of the most purchased color for 2018
#customers_articles_2018 = articles_transactions.query('t_dat < "2019-01-01"')
#customers_articles_2018 = customers_articles_2018.groupby(['customer_id'])['colour_group_code'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_articles_2018.rename(columns = {'colour_group_code':'top_colour_2018'}, inplace = True)
#print(customers_articles_2018.head())


In [ ]:
# Find the mode of the most purchased color for 2019
#customers_articles_2019 = articles_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
#customers_articles_2019 = customers_articles_2019.groupby(['customer_id'])['colour_group_code'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_articles_2019.rename(columns = {'colour_group_code':'top_colour_2019'}, inplace = True)


In [ ]:
# Find the mode of the most purchased color for 2020
#customers_articles_2020 = articles_transactions.query('t_dat >= "2020-01-01"')
#customers_articles_2020 = customers_articles_2020.groupby(['customer_id'])['colour_group_code'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_articles_2020.rename(columns = {'colour_group_code':'top_colour_2020'}, inplace = True)


In [ ]:
# Merge these new df in the customers table
#customers = pd.merge(customers, customers_articles_2018, on ='customer_id',how="left")
#customers = pd.merge(customers, customers_articles_2019, on ='customer_id',how="left")
#customers = pd.merge(customers, customers_articles_2020, on ='customer_id',how="left")

In [ ]:
#print(customers.head())
#print(customers.shape)

In [ ]:
#print(customers['top_colour_2019'].unique())

#print(articles[articles['colour_group_code']== -1])

In [ ]:
# Find the mode of the most purchased garment for 2018
#customers_garments_2018 = articles_transactions.query('t_dat < "2019-01-01"')
#customers_garments_2018 = customers_garments_2018.groupby(['customer_id'])['garment_group_no'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_garments_2018.rename(columns = {'garment_group_no':'top_garment_2018'}, inplace = True)

In [ ]:
# Find the mode of the most purchased garment for 2019
#customers_garment_2019 = articles_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
#customers_garment_2019 = customers_garment_2019.groupby(['customer_id'])['garment_group_no'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_garment_2019.rename(columns = {'garment_group_no':'top_garment_2019'}, inplace = True)

In [ ]:
# Find the mode of the most purchased garment for 2020
#customers_garment_2020 = articles_transactions.query('t_dat >= "2020-01-01"')
#customers_garment_2020 = customers_garment_2020.groupby(['customer_id'])['garment_group_no'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_garment_2020.rename(columns = {'garment_group_no':'top_garment_2020'}, inplace = True)

In [ ]:
# Merge these new df in the customers table
#customers = pd.merge(customers, customers_garments_2018, on ='customer_id',how="left")
#customers = pd.merge(customers, customers_garment_2019, on ='customer_id',how="left")
#customers = pd.merge(customers, customers_garment_2020, on ='customer_id',how="left")
#print(customers.head())

In [ ]:
print(customers.shape)

In [ ]:
# Find the mode of the most visited section for 2018
#customers_section_2018 = articles_transactions.query('t_dat < "2019-01-01"')
#customers_section_2018 = customers_section_2018.groupby(['customer_id'])['section_no'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_section_2018.rename(columns = {'section_no':'top_section_2018'}, inplace = True)

In [ ]:
# Find the mode of the most visited section for 2019
#customers_section_2019 = articles_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
#customers_section_2019 = customers_section_2019.groupby(['customer_id'])['section_no'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_section_2019.rename(columns = {'section_no':'top_section_2019'}, inplace = True)

In [ ]:
# Find the mode of the most visited section for 2020
#customers_section_2020 = articles_transactions.query('t_dat >= "2020-01-01"')
#customers_section_2020 = customers_section_2020.groupby(['customer_id'])['section_no'].apply(lambda x: x.mode().iloc[0]).to_frame()
#customers_section_2020.rename(columns = {'section_no':'top_section_2020'}, inplace = True)

In [ ]:
#customers = pd.merge(customers, customers_section_2018, on ='customer_id',how="left")
#customers = pd.merge(customers, customers_section_2019, on ='customer_id',how="left")
#customers = pd.merge(customers, customers_section_2020, on ='customer_id',how="left")
#print(customers.head())

In [ ]:
#customers.to_csv("customers_complete_numeric.csv")

In [ ]:
# Replace names
# I replace numeric categories with categorical attributes
#customers['club_member_status'].replace({'ACTIVE':1 },inplace=True)
#customers['club_member_status'].replace({'PRE-CREATE':2},inplace=True)
#customers['club_member_status'].replace({'LEFT CLUB':0},inplace=True)

#customers['fashion_news_frequency'].replace({'None': 0},inplace=True)
#customers['fashion_news_frequency'].replace({'Regularly': 1},inplace=True)
#customers['fashion_news_frequency'].replace({'Monthly':2 },inplace=True)

In [ ]:
#customers.to_csv("customers_complete_numeric.csv")

In [ ]:
print(customers['Active'].unique(), customers['FN'].unique(), customers['club_member_status'].unique(),customers['fashion_news_frequency'].unique() )

In [ ]:
# I would drop FN because only 84 customers (0.03%) that have fashion_news_frequency==None have FN==1, but I don't
#rslt_df = customers['fashion_news_frequency'][customers['FN'] == 1]
#print(rslt_df[rslt_df == 'None'].count())
#print(customers.shape)

In [ ]:
print(customers.dtypes)

In [ ]:
## Try to create a new feature, instead of count colors i try to make count perceived/ count colours
customers_perceived_2018 = articles_transactions.query('t_dat < "2019-01-01"')
customers_perceived_2018 = customers_perceived_2018.filter(['customer_id', 'perceived_colour_master_id'])
customers = pd.merge(customers,customers_perceived_2018.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'perceived_colour_master_id':'count_perceived_2018'}, inplace = True)

customers_perceived_2019 = articles_transactions.query('t_dat < "2020-01-01" & t_dat>="2019-01-01"')
customers_perceived_2019 = customers_perceived_2019.filter(['customer_id', 'perceived_colour_master_id'])
customers = pd.merge(customers,customers_perceived_2019.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'perceived_colour_master_id':'count_perceived_2019'}, inplace = True)

customers_perceived_2020 = articles_transactions.query('t_dat >= "2020-01-01"')
customers_perceived_2020 = customers_perceived_2020.filter(['customer_id', 'perceived_colour_master_id'])
customers = pd.merge(customers,customers_perceived_2020.groupby('customer_id').nunique(), on='customer_id')
customers.rename(columns = {'perceived_colour_master_id':'count_perceived_2020'}, inplace = True)
print(customers.head())

In [ ]:
# Add column related to the ratio of number of perceived colours/number of colours
print(customers.keys())
customers['ratio_perc_concrete_2018']= ""
customers['ratio_perc_concrete_2018'] = customers['count_perceived_2018']/customers['count_colours_2018'] 
print(customers.head())

customers['ratio_perc_concrete_2019']= ""
customers['ratio_perc_concrete_2019'] = customers['count_perceived_2019']/customers['count_colours_2019'] 
print(customers.head())

customers['ratio_perc_concrete_2020']= ""
customers['ratio_perc_concrete_2020'] = customers['count_perceived_2020']/customers['count_colours_2020'] 
print(customers.head())

In [ ]:
# Drop count perceived because they were helpful only to calculate the ratio
#customers = customers.drop("count_perceived_2018", 1)
#customers = customers.drop("count_perceived_2019", 1)
#customers = customers.drop("count_perceived_2020", 1)

In [ ]:
# I CREATE THE FEATURE COUNT GARMENTS/TOTAL GARMENTS INSTEAD OF COUNT GARMENTS ONLY (to remove from here)
tot_garments= len(articles['garment_group_no'].unique())
customers['ratio_garment_total_2018']=""
customers['ratio_garment_total_2019']=""
customers['ratio_garment_total_2020']=""
customers['ratio_garment_total_2018']= customers['count_garments_2018']/customers['tot_articles_2018']
customers['ratio_garment_total_2019']= customers['count_garments_2019']/customers['tot_articles_2019']
customers['ratio_garment_total_2020']= customers['count_garments_2020']/customers['tot_articles_2020']


#customers = customers.drop("count_garments_2018_zscore", 1)
#customers = customers.drop("count_garments_2019_zscore", 1)
#customers = customers.drop("count_garments_2020_zscore", 1)

In [ ]:
# I average the avg articles and the avg amount
customers['avg_amount']=""
customers['avg_articles']=""
customers['avg_amount'] = customers['avg_amount_2018']+customers['avg_amount_2019']+customers['avg_amount_2020']/3
customers['avg_articles'] = customers['avg_articles_2018']+customers['avg_articles_2019']+customers['avg_articles_2020']/3
customers.head()

In [ ]:
# Perform z-score normalization on variables that are not one hot encoded
cols = customers.columns
cols = cols.drop("customer_id", 1)
cols = cols.drop("fashion_news_frequency", 1)
cols = cols.drop("FN", 1)
cols = cols.drop("club_member_status", 1)
cols = cols.drop("Active", 1)
cols=list(cols)
print(cols)

customers_norm=pd.DataFrame(customers)   
# now iterate over the remaining columns and create a new zscore column
for col in cols:
    col_zscore = col + '_zscore'
    customers_norm[col_zscore] = (customers_norm[col] - customers_norm[col].mean())/customers_norm[col].std(ddof=0)

In [ ]:
# drop columns that are not normalized
for col in cols:
    if '_zscore' not in col:
        customers_norm = customers_norm.drop(col, 1)
    else:
        print('')

In [ ]:
# One hot encoding of Active variable

df = pd.DataFrame(list(zip(customers_norm['customer_id'], customers_norm['Active'].unique())),
                  columns=['customer_id', 'Active'])

y = pd.get_dummies(customers_norm['Active'], prefix='')
y.rename(columns = {'_0.0':'Non-Active'}, inplace = True)
y.rename(columns = {'_1.0':'Active'}, inplace = True)
y = y.astype({"Active": "float64"}, errors='raise') 
y = y.astype({"Non-Active": "float64"}, errors='raise') 
y.insert(0, 'customer_id', customers['customer_id'])

customers_norm = customers_norm.drop("Active", 1)
customers_norm = pd.merge(customers_norm, y, on='customer_id', how ='left')


In [ ]:
# One hot encoding of FN variable

df = pd.DataFrame(list(zip(customers_norm['customer_id'], customers_norm['FN'].unique())),
                  columns=['customer_id', 'FN'])

y = pd.get_dummies(customers_norm['FN'], prefix='')
y.rename(columns = {'_0.0':'FN_no'}, inplace = True)
y.rename(columns = {'_1.0':'FN_yes'}, inplace = True)
y = y.astype({"FN_yes": "float64"}, errors='raise') 
y = y.astype({"FN_no": "float64"}, errors='raise') 
y.insert(0, 'customer_id', customers_norm['customer_id'])

customers_norm = customers_norm.drop("FN", 1)
customers_norm = pd.merge(customers_norm, y, on='customer_id', how ='left')
print(customers_norm.head())

In [ ]:
# One hot encoding of club_member_status variable

df = pd.DataFrame(list(zip(customers_norm['customer_id'], customers_norm['club_member_status'].unique())),
                  columns=['customer_id', 'club_member_status'])

y = pd.get_dummies(customers_norm['club_member_status'], prefix='')
y.rename(columns = {'_ACTIVE':'club_status_active'}, inplace = True)
y.rename(columns = {'_PRE-CREATE':'club_status_precreate'}, inplace = True)
y.rename(columns = {'_LEFT CLUB':'club_status_leftclub'}, inplace = True)
y = y.astype({"club_status_active": "float64"}, errors='raise') 
y = y.astype({"club_status_precreate": "float64"}, errors='raise')
y = y.astype({"club_status_leftclub": "float64"}, errors='raise') 
y.insert(0, 'customer_id', customers_norm['customer_id'])

customers_norm = customers_norm.drop("club_member_status", 1)
customers_norm = pd.merge(customers_norm, y, on='customer_id', how ='left')
print(customers_norm.head())

In [ ]:
# One hot encoding of fashion_news_frequency variable

df = pd.DataFrame(list(zip(customers_norm['customer_id'], customers_norm['fashion_news_frequency'].unique())),
                  columns=['customer_id', 'fashion_news_frequency'])

y = pd.get_dummies(customers_norm['fashion_news_frequency'], prefix='')
y.rename(columns = {'_None':'fn_frequency_none'}, inplace = True)
y.rename(columns = {'_Regularly':'fn_frequency_regularly'}, inplace = True)
y.rename(columns = {'_Monthly':'fn_frequency_monthly'}, inplace = True)
y = y.astype({"fn_frequency_none": "float64"}, errors='raise') 
y = y.astype({"fn_frequency_regularly": "float64"}, errors='raise') 
y = y.astype({"fn_frequency_monthly": "float64"}, errors='raise') 
y.insert(0, 'customer_id', customers['customer_id'])

customers_norm = customers_norm.drop("fashion_news_frequency", 1)
customers_norm = pd.merge(customers_norm, y, on='customer_id', how ='left')
print(customers_norm.head())

In [ ]:
print(customers_norm.dtypes)

In [ ]:
#customers.to_csv("customers_numeric.csv")

In [ ]:
mean = statistics.fmean(customers_norm["age_zscore"])
print(mean)
stdev = statistics.stdev(customers_norm["age_zscore"])
print(stdev)
#variance = statistics.variance(transactions["price"])
#print(variance)
#norm = ln.norm(transactions["price"])
#print(norm)

In [ ]:
#customers = customers.drop("customer_id", 1)

In [ ]:
print(customers_norm.head())

In [ ]:
print(customers_norm.dtypes)

In [ ]:
# Matrice di correlazione
corr_df = customers_norm.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
#corr_df.to_csv("correlation_matrix.csv")

In [ ]:
### AFTER THIS SHITTY ANALYSIS I DECIDE TO DROP ONE BETWEEN COUNT AND AVERAGE (FOR AMOUNT AND ARTICLES)

customers_1 = customers_norm.drop("tot_articles_2018_zscore", 1)
customers_1 = customers_1.drop("tot_articles_2019_zscore", 1)
customers_1 = customers_1.drop("tot_articles_2020_zscore", 1)
customers_1 = customers_1.drop("tot_amount_2018_zscore", 1)
customers_1 = customers_1.drop("tot_amount_2019_zscore", 1)
customers_1 = customers_1.drop("tot_amount_2020_zscore", 1)

In [ ]:
# AFTER HAVING INTRODUCED THE RATIO, I REMOVE THE COUNT COLOURS
customers_1 = customers_1.drop("count_colours_2018_zscore", 1)
customers_1 = customers_1.drop("count_colours_2019_zscore", 1)
customers_1 = customers_1.drop("count_colours_2020_zscore", 1)

In [ ]:
# Matrice di correlazione
corr_df = customers_1.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
"""# I CREATE THE FEATURE COUNT GARMENTS/TOTAL GARMENTS INSTEAD OF COUNT GARMENTS ONLY
tot_garments= len(articles['garment_group_no'].unique())
print(tot_garments)
customers['ratio_garment_total_2018']=""
customers['ratio_garment_total_2019']=""
customers['ratio_garment_total_2020']=""
customers['ratio_garment_total_2018']= customers['count_garments_2018_zscore']/tot_garments
customers['ratio_garment_total_2019']= customers['count_garments_2019_zscore']/tot_garments
customers['ratio_garment_total_2020']= customers['count_garments_2020_zscore']/tot_garments"""

In [ ]:
customers_2 = customers_1.drop("count_garments_2018_zscore", 1)
customers_2 = customers_2.drop("count_garments_2019_zscore", 1)
customers_2 = customers_2.drop("count_garments_2020_zscore", 1)
customers_2 = customers_2.drop("count_perceived_2018_zscore", 1)
customers_2 = customers_2.drop("count_perceived_2019_zscore", 1)
customers_2 = customers_2.drop("count_perceived_2020_zscore", 1)

In [ ]:
# Matrice di correlazione
corr_df = customers_2.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
customers_2 = customers_2.drop("avg_articles_2018_zscore", 1)
customers_2 = customers_2.drop("avg_articles_2019_zscore", 1)
customers_2 = customers_2.drop("avg_articles_2020_zscore", 1)
customers_2 = customers_2.drop("avg_amount_2018_zscore", 1)
customers_2 = customers_2.drop("avg_amount_2019_zscore", 1)
customers_2 = customers_2.drop("avg_amount_2020_zscore", 1)

In [ ]:
# Matrice di correlazione
corr_df = customers_2.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
print(customers_2.dtypes)

In [ ]:
#print(customers.filter(['sales_channel_2_zscore', 'Woman_zscore','Young_zscore']))

In [ ]:
# Discovering correlations...why sales_channel_2 is correlated 

import seaborn as sns

temp = transactions.groupby(["sales_channel_id"])["customer_id"].count()
print(temp.index, temp.values)
df = pd.DataFrame({"sales_channel": temp.index, "Customers": temp.values})
df = df.sort_values(["sales_channel"], ascending=False)
plt.figure(figsize=(16, 5))
plt.title("Number of customers for sales channel")
s = sns.barplot(x="sales_channel", y="Customers", data=df, palette="cubehelix")
s.set_xticklabels(s.get_xticklabels(), rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
# Plot the correlation between Woman count and transaction with sales_channel_2

customers_norm.plot(kind = 'scatter', x = 'sales_channel_2_zscore', y = 'Woman_zscore')

plt.show()

#Features are very correlated

In [ ]:
# Plot correlation between sales_channel_2 and Young count

customers_norm.plot(kind = 'scatter', x = 'sales_channel_2_zscore', y = 'Young_zscore')

plt.show()

In [ ]:
# Plot correlation between avg_amount_zscore and avg_articles_zscore

customers_norm.plot(kind = 'scatter', x = 'avg_amount_zscore', y = 'avg_articles_zscore')

plt.show()

In [ ]:
# I begin deleting sales_channel_2 and one hot encoded that are very correlated (Non-Active, FN_no)(fn_frequency_none, Non-Active)
# (FN_yes, fn_frequency_regularly)
customers_3 = customers_2.drop("sales_channel_2_zscore", 1)
customers_3 = customers_3.drop("FN_no", 1)
customers_3 = customers_3.drop("fn_frequency_none", 1)
customers_3 = customers_3.drop("FN_yes", 1)
customers_3 = customers_3.drop("Active", 1)
customers_3 = customers_3.drop("Non-Active", 1)
#customers=customers.drop('avg_amount_zscore',1)

In [ ]:
# Matrice di correlazione
corr_df = customers_3.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
customers_3=customers_3.drop('avg_articles_zscore',1)

corr_df = customers_3.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
customers_4=customers_3.drop('avg_amount_zscore', 1)

corr_df = customers_4.corr(method='pearson')
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
## From this 18 features I apply clustering K Means with Elbow Method
from sklearn.cluster import KMeans

print(customers_4.dtypes)

customers_5 = customers_4.drop ('customer_id', 1)

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(customers_5)
    distortions.append(kmeanModel.inertia_)



In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
customers_5.head()

In [ ]:
# The number of clusters is 3, before the curve flattens
kmeans_model = KMeans(n_clusters = 3)

label = kmeans_model.fit_predict(customers_5.to_numpy())

centroids = kmeans_model.cluster_centers_



In [ ]:
#2-dimensional plotting on two random dimensions
'''u_labels = np.unique(label)

print(u_labels)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(customers_4.to_numpy()[label == i , 'club_status_active'] , customers_4.to_numpy()[label == i , 'age_zscore'] , label = i, alpha=0.5)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k', alpha=0.5)
plt.legend()
plt.show()'''

In [ ]:
# We want a multidimensional view on our data, so we plot parallel coodrinates

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

scaled_df = ss.fit_transform(customers_5.to_numpy())
scaled_df = pd.DataFrame(scaled_df)
customers_4["cluster"] = kmeans_model.labels_
final_df = pd.concat([scaled_df, customers_4['cluster']], axis=1)


In [ ]:
# plot parallel coordinates
from pandas.plotting import parallel_coordinates
pc = parallel_coordinates(final_df, 'cluster', color=('#FFE888', '#FF9999','#3302ba'))

In [ ]:
# Print cohordinates of the centroids

print(kmeans_model.cluster_centers_)

In [ ]:
# Plot histograms that represent the distribution of the cluster in respect to each feature

# How many instances do we have for each cluster?
print(customers_4.head())
temp = customers_4.groupby(["cluster"])["customer_id"].count()
print(temp.index, temp.values)
df = pd.DataFrame({"cluster": temp.index, "customers": temp.values})
df = df.sort_values(["cluster"], ascending=False)
plt.figure(figsize=(16, 5))
plt.title("Number of customers for sales channel")
s = sns.barplot(x="cluster", y="customers", data=df, palette="cubehelix")
s.set_xticklabels(s.get_xticklabels(), rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
# Since visualizing the clusters in this way is very low informative, i need to select the visualization of clusters among the
# most promising features

#filter rows of original data
'''filtered_label0 = customers_4[label == 0]
 
#plotting the results
plt.scatter(filtered_label0[:,0] , filtered_label0[:,1])
plt.show()
'''

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
customers_6=customers_4.drop('customer_id',1)

silhouette_avg = silhouette_score(customers_6, customers_6['cluster'])
print(silhouette_avg)

# 0.1495051505822442 is the silhouette score without dimensionality reduction, with 3 clusters. Is near to 0, so clusters overlap.

In [ ]:
customers.head()

cluster_df = pd.concat([
   customers, 
   customers_6['cluster']], 
axis=1)

for c in customers_6.columns[1:]:
    grid = sns.FacetGrid(cluster_df, col='cluster', height=3, aspect=1.3)
    grid.map(plt.hist, c, bins=20, edgecolor='k')
    grid.set_xticklabels(rotation=40)

In [ ]:
'''def prepare_pca(n_components, data, kmeans_labels):
    names = ['x', 'y', 'z']
    matrix = PCA(n_components=n_components).fit_transform(data)
    df_matrix = pd.DataFrame(matrix)
    df_matrix.rename({i:names[i] for i in range(n_components)}, axis=1, inplace=True)
    df_matrix['labels'] = kmeans_labels
    
    return df_matrix'''

In [ ]:
'''# Now I try performing the PCA and check if we can find more information about clusters that we have 
customers_5 = customers_4.drop('cluster',1)
pca_df = prepare_pca(2, customers_5, kmeans_model.labels_)
sns.scatterplot(x=pca_df.x, y=pca_df.y, hue=pca_df.labels, 
                palette="Set2")'''

In [ ]:
'''
pca = PCA(.95)
principalComponents = pca.fit_transform(customers_5)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])'''